In [2]:
%load_ext nb_mypy

Version 1.0.5


# Problem with TypeAliases

In [6]:
from typing import Generic, TypeVar, TypeAlias, Any, final
from abc import abstractmethod

from sagemaker.processing import Processor
from sagemaker.estimator import EstimatorBase

from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep, \
    TuningStep, ConfigurableRetryStep

# todo: decide how stringent this should be while still allowing user to add new step types
StepType = TypeVar("StepType", bound=ConfigurableRetryStep)
StepActor: TypeAlias = Processor |  EstimatorBase  # todo: add more types as needed
ConcreteStep: TypeAlias = ProcessingStep | TrainingStep | CreateModelStep | TransformStep | TuningStep


class StepFactoryInterface(Generic[StepType]):
    # @abstractmethod
    # def __init__(self, step: StepType):
    #     self.step_cls = StepType

    @property
    @abstractmethod
    # Note: This needs to return a *concrete* subclass of ConfigurableRetryStep
    def step_cls(self) -> ConcreteStep:
        ...

    def step_type(self) -> str:
        map = {
            ProcessingStep: 'processor',
        }

        return map[self.step_cls]

    @abstractmethod
    def _instantiate_step_actor(self) -> StepActor:
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        ...

    @abstractmethod
    def _construct_run_args(self) -> dict[str, Any]:  # todo: create dataclass for return types
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        ...

    def type(self):
        return StepType

    @final
    def create_step(self) -> StepType:
        step_actor: StepActor = self._instantiate_step_actor()
        run_args: dict[str, Any] = self._construct_run_args()
        # Type checker complains because you can't instantiate a ConfigurableRetryStep directly
        # (it's abstract). Ignore for now, as I don't think it's easily possible to specify we're
        # passing a concrete subclass.
        return self.step_cls(
            **{step: step_actor},
            **run_args
        )

ProcessingStepFactoryInterface = StepFactoryInterface[ProcessingStep]
psf = ProcessingStepFactoryInterface()

<cell>32: error: Invalid index type "ProcessingStep | TrainingStep | CreateModelStep | TransformStep | TuningStep" for "dict[type[ProcessingStep], str]"; expected type "type[ProcessingStep]"  [index]
<cell>54: error: "ProcessingStep" not callable  [operator]
<cell>54: error: "TrainingStep" not callable  [operator]
<cell>54: error: "CreateModelStep" not callable  [operator]
<cell>54: error: "TransformStep" not callable  [operator]
<cell>54: error: "TuningStep" not callable  [operator]
<cell>55: error: Name "step" is not defined  [name-defined]
<cell>60: error: Cannot instantiate abstract class "BaseStepFactoryInterface" with abstract attributes "_construct_run_args", "_instantiate_step_actor" and "step_cls"  [abstract]


# Trying virtual subclasses


In [13]:
from typing import Generic, TypeVar, TypeAlias, Any, final
from abc import ABC, abstractmethod

from sagemaker.processing import Processor
from sagemaker.estimator import EstimatorBase

from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep, \
    TuningStep, ConfigurableRetryStep

StepActor: TypeAlias = Processor |  EstimatorBase  # todo: add more types as needed
class ConcreteStep(ABC):
    ...

ConcreteStep.register(ProcessingStep)
ConcreteStep.register(TrainingStep)
ConcreteStep.register(CreateModelStep)
ConcreteStep.register(TransformStep)
ConcreteStep.register(TuningStep)

# todo: decide how stringent this should be while still allowing user to add new step types
StepType = TypeVar("StepType", bound=ConcreteStep)
class StepFactoryInterface(Generic[StepType]):
    @abstractmethod
    def _instantiate_step_actor(self) -> StepActor:
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        ...

    @abstractmethod
    def construct_run_args(self) -> dict[str, Any]:  # todo: create dataclass for return types
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        # Changed: needs to include stepactor, e.g. processor=processor
        ...

    @final
    def create_step(self) -> StepType:
        step_actor: StepActor = self._instantiate_step_actor()
        run_args: dict[str, Any] = self.construct_run_args()
        # Type checker complains because you can't instantiate a ConfigurableRetryStep directly
        # (it's abstract). Ignore for now, as I don't think it's easily possible to specify we're
        # passing a concrete subclass.
        return StepType(
            **run_args
        )

ProcessingStepFactoryInterface = StepFactoryInterface[ProcessingStep]
# psf = ProcessingStepFactoryInterface()

<cell>41: error: "StepType" is a type variable and only valid in type context  [misc]
<cell>45: error: Type argument "ProcessingStep" of "BaseStepFactoryInterface" must be a subtype of "ConcreteStep"  [type-var]
<cell>45: error: Value of type variable "StepType" of "BaseStepFactoryInterface" cannot be "ProcessingStep"  [type-var]
<cell>46: error: Value of type variable "StepType" of "BaseStepFactoryInterface" cannot be "ProcessingStep"  [type-var]
<cell>46: error: Cannot instantiate abstract class "BaseStepFactoryInterface" with abstract attributes "_instantiate_step_actor" and "construct_run_args"  [abstract]


# Only create basic interface

In [16]:
from typing import Generic, TypeVar, TypeAlias, Any, final
from abc import abstractmethod

from sagemaker.processing import Processor
from sagemaker.estimator import EstimatorBase

from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep, \
    TuningStep, ConfigurableRetryStep

# todo: decide how stringent this should be while still allowing user to add new step types
StepType = TypeVar("StepType", bound=ConfigurableRetryStep)


class StepFactoryInterface(Generic[StepType]):
    @abstractmethod
    def create_step(self) -> StepType:
        ...

ProcessingStepFactoryInterface = StepFactoryInterface[ProcessingStep]


Note: by contrast to earlier designs, create_step does not require a step_config to be passed here as an argument. This simplify the interface considerably because we don't have to worry about getting the right type of step_config for a given step_type. The problem is that while we can simply *return more specific* step types in subclasses (such as a ProcessingStep instead of a ConfigurableRetryStep), we can**not** require a *more specific argument* for subclasses (for example requiring a ProcessingStepConfig for a ProcessingStepFactory), as this would violate the Liskov Substitution Principal.

This problem would not even be easily solved by using generics, because it is not obvious how we can go from my given Steptype to the associated StepConfig.

In [ ]:
from sagemaker.session import Session
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

# IMPLEMENTATION
# ===============

class FrameworkProcessingStepFactory(ProcessingStepFactoryInterface):
    def __init__(
        self,
        step_name: str,
        config_facade_cls: type[FrameworkProcessingConfigFacade], # todo: generalize type
    ):
        self._step_name = step_name
        self._config_facade_cls = config_facade_cls

    def create_step(self) -> ProcessingStep:
        return ProcessingStep(
            **args,
            depends_on=self.config_facade._steptype_specific_config.depends_on,
        )

    def instantiate_config_facade(
        self,
        shared_config: SharedConfig,
        steptype_specific_config: ProcessingConfig,
        additional_config: FrameworkProcessingConfig,
    ):
        # problem: type checker doesn't know when this attributes exists
        self.config_facade = self._config_facade_cls(
            shared_config=shared_config,
            steptype_specific_config=steptype_specific_config,
            additional_config=additional_config,
        )

    @property
    def step_name(self) -> str:
        """
        Facade calls this method before create_step(), so it can retrieve the right config first.
        """
        return self._step_name

    # todo: Generalize types to other processors
    def _processor(
        self,
        step_config_facade: FrameworkProcessingConfigFacade,
        sagemaker_session: PipelineSession | Session | LocalPipelineSession,
        role_arn: str,
    ) -> Processor:  # type: ignore
        """Instantiate processor."""

        # Get processor class from classname string in config file

        return processor_cls(

        )  # type: ignore

    def _get_processor_run_args(
        self,
        step_config_facade: FrameworkProcessingConfigFacade,
    ) -> FrameworkProcessorRunArgs:

        def _construct_outputs(
            step_config_facade: FrameworkProcessingConfigFacade,
        ) -> list[ProcessingOutput]:
            """Constructs list of ProcessingOutput from configs."""
            outputs: list[ProcessingOutput] = []
            for output_config in step_config_facade.steptype_specific_config.outputs:
                output = ProcessingOutput(
                    source=f"/opt/ml/{self.step_type}/{step_config_facade.step_name}/{output_config.output_name}",
                    destination=f"{step_config_facade.s3_output_folder}/{output_config.output_name}",
                    **output_config,
                )
                outputs.append(output)
            return outputs

        skl_run_args = FrameworkProcessorRunArgs(
            inputs = [
                ProcessingInput(
                    source=step_config_facade.input_s3_dir,
                    destination=f'/opt/ml/{self.step_type}/{step_config_facade.step_name}/input/'
                ),
            ],
            outputs = _construct_outputs(step_config_facade),
            source_dir=f"code/{_step_config_facade.step_name}/",
            code=f'{step_config_facade.step_name}.py', # Hard-code to avoid extra config value
            arguments=None, # Todo: Decide whether this should come from configuration. May depend on type of step.
        )
        return skl_run_args

fw_proc_step_factory = FrameworkProcessingStepFactory(step_name='preprocessing')

# Simple factory
This makes better use of factory, because depending on arg passed to it, it creates a different type of step. Otherwise, we may as well us strategy pattern (only use of factory is to construct step later when configs etc are known - but a given factory always produces same kind of step, except from configuration).

In [5]:
from typing import ClassVar


class StepFactoryInterface(ABC):
    @abstractmethod
    def create_step(self, step_name: str) -> ConfigurableRetryStep:
        ...


class StepFactoryWrapper:
    def __init__(
        self,
        stepfactory_lookup_table: dict[str, StepFactoryInterface],
    ):
        self._stepfactory_lookup_table = stepfactory_lookup_table

    # def set_stepname_to_factory_map(
    #     self,
    #     stepname_to_factory_map: dict[str, StepFactoryInterface],
    # ):
    #     self._steptype_to_factory_map = stepname_to_factory_map

    # def update_stepname_to_factory_map(
    #     self,
    #     additional_stepname_to_factory_map: dict[str, StepFactoryInterface],
    # ):
    #     self._steptype_to_factory_map.update(
    #         additional_stepname_to_factory_map,
    #     )

    def create_step(self, step_name: str) -> ConfigurableRetryStep:
        # step name identifies config location.
        step_config: dict[str, Any] = load_step_config_dict_from_yaml(step_name=step_name)
        # Before validating config, we need to know for what type of step it is.
        # todo: Make knowable for typechecker that this key exists. Use typeddict (with optional keys)?
        step_type = step_config['step_type']
        # Look up which factory to use, based on step_type speified in config
        factory: StepFactoryInterface = self._stepfactory_lookup_table[step_type]
        return factory.create_step(step_config=step_config)


class _FrameworkProcessingStepFactory():
    def create_step(self, step_config: dict[str: Any]) -> ProcessingStep:
        ...

SyntaxError: invalid syntax (1812735520.py, line 39)

Note that the StepFactoryWrapper is decoupled from the specific StepFactory that will be used to create the step. The latter is determined by a lookup table, which is injected into to the StepFactoryWrapper during initialization.

The downside is that this is less convenient for simple use cases, where the user is content with choosing only from the default factories that ship with the library. To remediate this disadvantage, we can simply create a facade, which instantiates the StepFactoryWrapper with the default lookup table. More advanced users, by contrast, can directly import this default lookup table and customize it to point to custom StepFactory implementations. In a second step, they then initialize the StepFactoryWrapper directly, passing it the custom lookup table.

In [ ]:
# higher-level-interface
# ======================

stepfactory_lookup_table: dict[str, StepFactoryInterface] = {
    'FrameworkProcessor': _FrameworkProcessingStepFactory,
}

# This is what user will import
stepfactory_wrapper = StepFactoryWrapper(
    stepfactory_lookup_table=stepfactory_lookup_table,
)

In [ ]:
# lower-level interface (if customization of factories is needed)
# ===============================================================

# Implement custom stepfactory
class  _CustomProcessingStepFactory():
    ...

# add it to the lookup table
stepfactory_lookup_table.update(
    {
        'CustomProcessor': _CustomProcessingStepFactory,
    },
)

# Instantiate StepFactory with customized lookup table
customized_step_factory = StepFactoryWrapper(
    stepfactory_lookup_table=stepfactory_lookup_table
)